In [ ]:
!pip install -U datasette

# SQLite intro

In [ ]:
import sqlite3
from pathlib import Path

import requests

pwd = Path('.').absolute() / 'datasette-demo'
db = sqlite3.connect(pwd / 'content.db')

print(db.execute('select sqlite_version()').fetchall())

for row in db.execute('SELECT * FROM plugin_repos LIMIT 10'):
    print(row)

db.row_factory = sqlite3.Row


def q(stmt):
    return [dict(row) for row in db.execute(stmt).fetchall()]


# List tables
q("SELECT name FROM sqlite_master where type='table'")

In [ ]:
db.execute("""
create table peps (
  id integer primary key,
  title text,
  author text,
  status text,
  type text,
  created text,
  body text
);
""")


def parse_pep(s):
    intro, body = s.split('\n\n', 1)
    pep = {}
    current_key = None
    current_value = None
    for line in intro.split('\n'):
        # If the line starts with whitespace, it's a continuation of the previous value
        if line.startswith(' ') or line.startswith('\t'):
            if current_key is not None:
                current_value += ' ' + line.strip()
                pep[current_key] = current_value.strip()
        else:
            # Split the line into key and value
            parts = line.split(': ', 1)
            if len(parts) == 2:
                key, value = parts
                # Update the current key and value
                current_key = key
                current_value = value
                # Add the key-value pair to the pep dictionary
                pep[current_key] = current_value.strip()
    pep['Body'] = body.strip()
    return pep

In [ ]:
zen = requests.get('https://raw.githubusercontent.com/python/peps/main/pep-0020.txt').text
pep = parse_pep(zen)

db.execute(
    'insert into peps (id, title, author, status, type, created, body)'
    '  values (:PEP, :Title, :Author, :Status, :Type, :Created, :Body)',
    pep,
)

q('SELECT * FROM peps')

# Datasette intro